In [31]:
import pandas as pd
from sqlalchemy import create_engine

# Extract

In [61]:
# engine = create_engine('sqlite:///:memory:', echo=True)
df = pd.read_csv('./app/data/DATA.csv')
df.head()

,id,first_name,last_name,email,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,Non-binary,49.55.218.81


# Transform

In [48]:
# find unique genders
unique_genders = df['gender'].unique()

# create genders dictionary to efficiently create gender_id column
genders = {}
for i, gender, in enumerate(unique_genders):
    genders[gender] = i + 1

# create gender_ids list to add to df
gender_ids = []
for gender in df['gender'].array:
    gender_ids.append(genders[gender])

# add gender_id column
df['gender_id'] = gender_ids

# create genders df table
df_genders = df[['gender_id', 'gender']].drop_duplicates()

# create people df table by dropping gender column
df_people = df.drop(['gender'], axis=1)

df_people.head()
# df_genders.head(100)


,id,first_name,last_name,email,ip_address,gender_id
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,34.148.232.131,1
1,2,Vally,Garment,vgarment1@wisc.edu,15.158.123.36,2
2,3,Tessa,Curee,tcuree2@php.net,132.209.143.225,2
3,4,Arman,Heineking,aheineking3@tuttocitta.it,157.110.61.233,3
4,5,Roselia,Trustie,rtrustie4@ft.com,49.55.218.81,4


# Load

In [60]:
# connect to postgres container from outside docker
db_name = 'database'
db_user = 'username'
db_pass = 'secret'
db_host = 'localhost'
db_port = '5433'

# Connect to the database
db_string = 'postgres://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

# load database
df.to_sql('raw_people', con=db, if_exists="replace", index=False)
df_genders.to_sql('genders', con=db, if_exists='replace', index=False)
df_people.to_sql('people', con=db, if_exists='replace', index=False)
